In [ ]:
from pathlib import Path
import re
import numpy as np
import pandas as pd

# ========================= Configuration =========================
# Table 1: Project Investment (Long format): year / region / cost
PATH_INVEST = Path( "./load and costs/MISO/MISO_cost.xlsx" )
COL_YEAR_1   = "year"
COL_REGION_1 = "region"
COL_COST_1   = "cost"

# Table 2: Regional Sub-tables; Columns must include Year / Residential Growth / DC Growth
PATH_GROWTH = Path( "./load and costs/MISO/22_load_compare.xlsx" )
# Table 3: Regional Sub-tables; Columns must include Year / RU Share / DC Share
PATH_SHARE  = Path( "./rider/MISO/00_MISO_share.xlsx" )

# Output Directory
OUTDIR = Path( "./rider/MISO" )
OUTDIR.mkdir(parents=True, exist_ok=True)

# Lookahead Window
LOOKAHEAD = 5  # 5-Year Forward Rolling Sum

# Column Alias Mapping (Normalized -> Standard)
# Adjust these based on your actual Excel headers
KEY_MAP = {
    "year": "year",
    "年份": "year",
    "region": "region",
    "zone": "region",
    "area": "region",
    "cost": "cost",
    "totalcost": "cost",
    "investment": "cost",
    "resdelta": "res_delta",
    "residentialdelta": "res_delta",
    "res": "res_delta",
    "dcdelta": "dc_delta",
    "dc_delta": "dc_delta",
    "dc": "dc_delta",
    "ru": "ru_share",
    "residential": "ru_share",
    "ru_share": "ru_share",
    "dc_share": "dc_share"
}

def _norm(s):
    """Normalize string: remove non-alphanumeric characters, convert to lowercase."""
    if pd.isna(s): return ""
    return re.sub(r"[^a-z0-9]+", "", str(s).lower())

def rename_cols(df):
    """Clean column names based on KEY_MAP."""
    new_cols = {}
    for c in df.columns:
        nc = _norm(c)
        if nc in KEY_MAP:
            new_cols[c] = KEY_MAP[nc]
    return df.rename(columns=new_cols)

# ========================= 1. Load Data =========================

# --- 1.1 Read Investment Table (Table 1) ---
df_inv = pd.read_excel(PATH_INVEST)
df_inv = rename_cols(df_inv)
# Ensure required columns exist
for c in ["year", "region", "cost"]:
    if c not in df_inv.columns:
        raise ValueError(f"Table 1 is missing column: {c}")
# Aggregation: One record per Region-Year
inv_agg = df_inv.groupby(["year", "region"], as_index=False)["cost"].sum()

# --- 1.2 Read Growth Table (Table 2) ---
# Read all sheets, assume each sheet is a Region
# Keep raw values (positive/negative), do not zero-floor yet
sheets_growth = pd.read_excel(PATH_GROWTH, sheet_name=None)
growth_list = []
for sheet_name, df in sheets_growth.items():
    df = rename_cols(df)
    if "year" not in df.columns:
        # Try to identify the first column as Year
        df = df.rename(columns={df.columns[0]: "year"})

    # Ensure Res/DC columns exist
    if "res_delta" not in df.columns or "dc_delta" not in df.columns:
        # Fallback: Identify by column index (1=Res, 3=DC) or keyword search?
        # Simple logic: If columns 1 and 3 are numeric, assume they are Res and DC
        # (Adjust logic based on actual file structure)
        cols = df.columns
        if len(cols) >= 4:
            # Assuming Col 1 is Res, Col 3 is DC (0-based indices 1 and 3)
            # This is risky, better to rely on keywords.
            pass

    df["region"] = sheet_name # Use sheet name as region
    growth_list.append(df[["year", "region", "res_delta", "dc_delta"]])

df_growth_raw = pd.concat(growth_list, ignore_index=True)
df_growth_raw["year"] = pd.to_numeric(df_growth_raw["year"], errors="coerce")
df_growth_raw = df_growth_raw.dropna(subset=["year"])
df_growth_raw["year"] = df_growth_raw["year"].astype(int)

# --- 1.3 Read Share Table (Table 3) ---
# Format: Each sheet is a Region. Rows are "RU", "DC". Columns are Years.
sheets_share = pd.read_excel(PATH_SHARE, sheet_name=None, index_col=0)
share_list = []
for sheet_name, df in sheets_share.items():
    # Transpose: Rows become Years, Columns become RU/DC
    df_t = df.T
    df_t.index.name = "year"
    df_t = df_t.reset_index()
    # Clean column names (Year, RU, DC)
    df_t = rename_cols(df_t)

    # Identify RU and DC columns in the transposed dataframe
    # The original index (now columns) might be "RU", "DC", "Residential" etc.
    # We need to map them to 'ru_share', 'dc_share'
    mapped_cols = {}
    for c in df_t.columns:
        nc = _norm(c)
        if "ru" in nc or "res" in nc:
            mapped_cols[c] = "ru_share"
        elif "dc" in nc:
            mapped_cols[c] = "dc_share"
        elif "year" in nc or str(c).isdigit(): # If year column wasn't handled
            mapped_cols[c] = "year"

    df_t = df_t.rename(columns=mapped_cols)
    df_t["region"] = sheet_name

    if "ru_share" in df_t.columns and "dc_share" in df_t.columns:
        share_list.append(df_t[["year", "region", "ru_share", "dc_share"]])

df_share = pd.concat(share_list, ignore_index=True)
df_share["year"] = pd.to_numeric(df_share["year"], errors="coerce")
df_share = df_share.dropna(subset=["year"])
df_share["year"] = df_share["year"].astype(int)

# ========================= 2. Calculation Logic =========================

# 2.1 Calculate Forward Rolling Sum
# Logic: For Year T, sum(LoadDelta[T : T+LOOKAHEAD])
# Implementation: Sort descending -> Rolling Sum -> Shift back?
# Or: Sort ascending -> Inverse rolling?
# Easier: Rolling on inverted index.

def calc_forward_rolling(df, val_col, window):
    # Sort by Year Ascending
    df = df.sort_values("year")
    # Invert, calculate rolling sum (looking "backward" which is forward in time), then invert back
    # min_periods=1 ensures we get values even at the end of the series
    rolling = df[val_col][::-1].rolling(window=window, min_periods=1).sum()[::-1]
    return rolling

# Calculate per region
df_growth_raw = df_growth_raw.sort_values(["region", "year"])
df_growth_raw["res_roll"] = df_growth_raw.groupby("region")\
                                         .apply(lambda x: calc_forward_rolling(x, "res_delta", LOOKAHEAD))\
                                         .reset_index(level=0, drop=True)
df_growth_raw["dc_roll"]  = df_growth_raw.groupby("region")\
                                         .apply(lambda x: calc_forward_rolling(x, "dc_delta",  LOOKAHEAD))\
                                         .reset_index(level=0, drop=True)

# 2.2 Load Increment Attribution Calculation
# Rule:
# 1. Zero-floor: If Rolling Sum < 0, treat as 0.
# 2. Calculate proportions:
#    - Denom = max(0, Res_Roll) + max(0, DC_Roll)
#    - If Denom == 0, then Rate = 0 (avoid div/0)
#    - Res_Rate = max(0, Res_Roll) / Denom
#    - DC_Rate  = max(0, DC_Roll)  / Denom

df_calc = df_growth_raw.copy()
df_calc["res_pos"] = df_calc["res_roll"].clip(lower=0)
df_calc["dc_pos"]  = df_calc["dc_roll"].clip(lower=0)
df_calc["denom"]   = df_calc["res_pos"] + df_calc["dc_pos"]

# Calculate internal allocation ratio (within the incremental part)
df_calc["ratio_res"] = np.where(df_calc["denom"] > 0, df_calc["res_pos"] / df_calc["denom"], 0.0)
df_calc["ratio_dc"]  = np.where(df_calc["denom"] > 0, df_calc["dc_pos"]  / df_calc["denom"], 0.0)

# 2.3 Merge Investment and Share Data
# Merge structure: Investment (Main) <- Growth Ratio <- RU/DC Share
# Note: Ensure Regions match.

# Standardize region names (uppercase, strip)
inv_agg["region"] = inv_agg["region"].astype(str).str.strip().str.upper()
df_calc["region"] = df_calc["region"].astype(str).str.strip().str.upper()
df_share["region"] = df_share["region"].astype(str).str.strip().str.upper()

# Merge Investment + Growth
merged = pd.merge(inv_agg, df_calc, on=["year", "region"], how="left")
# Fill missing growth data with 0 (implies no growth -> no specific attribution?)
# Or maybe fill ratios? If no growth data, we can't attribute by growth.
merged[["ratio_res", "ratio_dc"]] = merged[["ratio_res", "ratio_dc"]].fillna(0.0)

# Calculate Cost Attribution
merged["cost_to_res_growth"] = merged["cost"] * merged["ratio_res"]
merged["cost_to_dc_growth"]  = merged["cost"] * merged["ratio_dc"]
# Note: If denom=0, Cost remains unallocated by growth logic.
# You might want a fallback or just track it as "Unassigned".
merged["cost_unassigned"] = merged["cost"] - (merged["cost_to_res_growth"] + merged["cost_to_dc_growth"])

# Merge RU/DC Share (for final user burden)
# This share splits the "Growth Cost" further? Or splits the Total Cost?
# Usually, Sankey logic:
# Layer 1: Total Cost -> [Res Growth Portion] / [DC Growth Portion]
# Layer 2: [Res Growth Portion] -> RU (100%?) or RU/DC split?
#          [DC Growth Portion] -> DC (100%?)
# Based on previous scripts (PJM/CAISO), usually:
# - "Cost caused by DC Growth" is allocated to DC users?
# - Or allocated to Zone, then split by User Share?
# Let's check CAISO logic:
#   RU_charge_from_DCgrowth = RegionCost_DCgrowth * RU_share
#   DC_charge_from_DCgrowth = RegionCost_DCgrowth * DC_share
#   (Meaning even if DC caused the growth, the cost enters the pool and is shared by RU/DC based on load share)
# Let's apply this logic (Sankey Conservation).

merged = pd.merge(merged, df_share, on=["year", "region"], how="left")
# Fill missing shares (maybe defaults?)
merged[["ru_share", "dc_share"]] = merged[["ru_share", "dc_share"]].fillna(0.0)

# Final calculation
# 1. From Res Growth Cost -> Split to RU / DC
merged["paid_by_ru_from_res"] = merged["cost_to_res_growth"] * merged["ru_share"]
merged["paid_by_dc_from_res"] = merged["cost_to_res_growth"] * merged["dc_share"]

# 2. From DC Growth Cost -> Split to RU / DC
merged["paid_by_ru_from_dc"] = merged["cost_to_dc_growth"] * merged["ru_share"]
merged["paid_by_dc_from_dc"] = merged["cost_to_dc_growth"] * merged["dc_share"]

# 3. Unassigned (if any) -> Split? Or keep separate?
# Let's assume unassigned (no growth) follows general share
merged["paid_by_ru_from_unassigned"] = merged["cost_unassigned"] * merged["ru_share"]
merged["paid_by_dc_from_unassigned"] = merged["cost_unassigned"] * merged["dc_share"]

# Total Paid
merged["paid_total_ru"] = merged["paid_by_ru_from_res"] + merged["paid_by_ru_from_dc"] + merged["paid_by_ru_from_unassigned"]
merged["paid_total_dc"] = merged["paid_by_dc_from_res"] + merged["paid_by_dc_from_dc"] + merged["paid_by_dc_from_unassigned"]

# ========================= 3. Generate Output =========================

# 3.1 Intermediate Table (Level 2: By Region)
# Cols: Year, Region, TotalCost, Cost_ResGrowth, Cost_DCGrowth, Cost_Unassigned
lev2 = merged[["year", "region", "cost", "cost_to_res_growth", "cost_to_dc_growth", "cost_unassigned"]].copy()
lev2.to_csv(OUTDIR/"miso_level2_alloc_by_growth.csv", index=False)

# 3.2 User Burden (Level 3)
lev3 = merged[["year", "region", "paid_total_ru", "paid_total_dc"]].copy()
lev3.to_csv(OUTDIR/"miso_level3_user_burden.csv", index=False)

# 3.3 Sankey Links
# Format: Source, Target, Value, Year
# Layers:
# L1: MISO_Invest -> Region_Total (Skip this? Usually Source -> GrowthType)
# Let's match CAISO/PJM style:
# Source: "MISO Grid" -> Target: "Region - ResGrowth" / "Region - DCGrowth"
# Then: "Region - ResGrowth" -> RU / DC
links = []

for _, row in merged.iterrows():
    y = row["year"]
    r = row["region"]

    # L1: Grid -> Growth Drivers
    # Node names
    node_grid = "MISO Grid"
    node_res_g = f"{r} ResGrowth"
    node_dc_g  = f"{r} DCGrowth"
    node_none  = f"{r} Unassigned"

    if row["cost_to_res_growth"] > 0:
        links.append({"year": y, "source": node_grid, "target": node_res_g, "value": row["cost_to_res_growth"]})
    if row["cost_to_dc_growth"] > 0:
        links.append({"year": y, "source": node_grid, "target": node_dc_g,  "value": row["cost_to_dc_growth"]})
    if row["cost_unassigned"] > 0:
        links.append({"year": y, "source": node_grid, "target": node_none,  "value": row["cost_unassigned"]})

    # L2: Growth Drivers -> Users (RU/DC)
    node_ru = f"{r} RU"
    node_dc = f"{r} DC"

    # From Res Growth
    if row["paid_by_ru_from_res"] > 0:
        links.append({"year": y, "source": node_res_g, "target": node_ru, "value": row["paid_by_ru_from_res"]})
    if row["paid_by_dc_from_res"] > 0:
        links.append({"year": y, "source": node_res_g, "target": node_dc, "value": row["paid_by_dc_from_res"]})

    # From DC Growth
    if row["paid_by_ru_from_dc"] > 0:
        links.append({"year": y, "source": node_dc_g, "target": node_ru, "value": row["paid_by_ru_from_dc"]})
    if row["paid_by_dc_from_dc"] > 0:
        links.append({"year": y, "source": node_dc_g, "target": node_dc, "value": row["paid_by_dc_from_dc"]})

    # From Unassigned
    if row["paid_by_ru_from_unassigned"] > 0:
        links.append({"year": y, "source": node_none, "target": node_ru, "value": row["paid_by_ru_from_unassigned"]})
    if row["paid_by_dc_from_unassigned"] > 0:
        links.append({"year": y, "source": node_none, "target": node_dc, "value": row["paid_by_dc_from_unassigned"]})

df_links = pd.DataFrame(links)
df_links.to_csv(OUTDIR/"miso_sankey_links.csv", index=False)

# ----------------- Check -----------------
# Verify totals
total_inv = inv_agg["cost"].sum()
total_allocated = df_links[df_links["source"]=="MISO Grid"]["value"].sum()
print(f"Total Investment: {total_inv:,.2f}")
print(f"Total Allocated (L1): {total_allocated:,.2f}")
print(f"Diff: {total_inv - total_allocated:,.2f}")

# Also check Level 2 + 3 combined output for Tableau
# Region | Year | Type (ResGrowth/DCGrowth/Unassigned) | Paid_RU | Paid_DC
# Melt and Merge
# (Simplified: Just use the Level 3 CSV or Links for visualization)

# Final National Burden (Raw Scale)
final_nat_raw = lev3.groupby("year", as_index=False)[["paid_total_ru", "paid_total_dc"]].sum()
final_nat_raw.to_csv(OUTDIR/"check_final_paid_national_raw.csv", index=False)

print("✅ All done. Output directory:", OUTDIR.resolve())

